In [1]:
# Insérez ici le numéro de votre équipe (mecbot'x')
VOTRE_NUMERO_EQUIPE = "xxx" # 1-12
VOTRE_NUMERO_DINGO = "3" # 1-8
# modifier cette variable selon que vous contrôler la simulation ou le robot réel
SIM = False

In [2]:
import socket
def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.settimeout(0)
    try:
        s.connect(('192.168.0.254', 1))
        IP = s.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        s.close()
    return IP
print(get_ip())

192.168.0.218


In [3]:
# Import des paquets requis et configuration du ROS Master
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/usr/lib/python3/dist-packages')
from os import environ
if SIM:
    environ['ROS_MASTER_URI'] = "http://localhost:11381/"
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-0{}.local:11311/".format(VOTRE_NUMERO_DINGO)
    environ['ROS_IP'] = get_ip()
import rospy
from std_msgs.msg import Float32MultiArray
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose
from sensor_msgs.msg import Imu

In [4]:
# Création d'un nouveau contenant ROSBAG (en écriture)
import rosbag

if SIM:
    ros_bag = rosbag.Bag('sim_bag.bag', 'w')
else:
    ros_bag = rosbag.Bag('real_bag.bag', 'w')

In [5]:
# Créer et démarrer un nouveau noeud
rospy.init_node('dingo_controller', anonymous=True)

In [6]:
# Wheel Position subscriber callback
wheel_pose_msg = Pose()
def wheel_pose_callback(msg):
    global wheel_pose_msg
    wheel_pose_msg  = msg.pose.pose

In [7]:
# Camera Position subscriber callback
cam_pose_msg = Pose()
def cam_pose_callback(msg):
    global cam_pose_msg
    cam_pose_msg  = msg.pose.pose

In [8]:
# IMU subscriber callback
imu_msg = Imu()
def imu_callback(msg):
    global imu_msg
    imu_msg = msg

In [9]:
# ROS subscribers et publishers

# SIM
if SIM:
    cmd_drive_pub_sim = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
    wheel_odom_sub_sim = rospy.Subscriber('mobile_manip/dingo_velocity_controller/odom', Odometry, wheel_pose_callback)
    cam_odom_sub_sim = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, cam_pose_callback)
    imu_sub_sim = rospy.Subscriber('/mobile_manip/imu/data', Imu, imu_callback)

# Robot
else:
    cmd_drive_pub_robot = rospy.Publisher('/mobile_manip/base/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
    wheel_odom_sub_robot = rospy.Subscriber('/mobile_manip/base/dingo_velocity_controller/odom', Odometry, wheel_pose_callback)
    cam_odom_sub_robot = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, cam_pose_callback)
    imu_sub_robot = rospy.Subscriber('/mobile_manip/base/imu/data', Imu, imu_callback)

In [10]:
# Impression des valeurs IMU et des encodeurs - pour tester
print("Imu data: ")
print("Orientation (quaternion): ", 
    imu_msg.orientation.x, imu_msg.orientation.y, 
    imu_msg.orientation.z, imu_msg.orientation.w
)
print("Angular velocity: ", 
    imu_msg.angular_velocity.x, imu_msg.angular_velocity.y, 
    imu_msg.angular_velocity.z
)
print("Linear acceleration: ", 
    imu_msg.linear_acceleration.x, imu_msg.linear_acceleration.y, 
    imu_msg.linear_acceleration.z
)
print("")
print("Wheel Position: ", 
    wheel_pose_msg.position.x, wheel_pose_msg.position.y
)
print("Camera Position: ", 
    cam_pose_msg.position.x, cam_pose_msg.position.y
)

Imu data: 
Orientation (quaternion):  -0.03148751729653795 -0.06499132636026336 0.9872700949227567 0.1417124674248705
Angular velocity:  -0.0048203058540821075 0.000916631892323494 -0.8929851055145264
Linear acceleration:  -0.4953858554363251 -1.2846405506134033 9.339807510375977

Wheel Position:  1.244251385599183 -0.14847746686589774
Camera Position:  0.2735408842563629 -1.0090152025222778


In [11]:
# Reprendre les dimensions du robot du projet1-2
interwheel_distance = 0.3765
left_wheel_radius = 0.098/2
right_wheel_radius = 0.098/2

In [12]:
# Déterminez les valeurs numériques de vitesse aux roues requises pour touner sur un cercle R=0.5m
cmd_drive_msg = Drive()

# Trouver les valeurs pour tourner sur un cercle de R=0.5m
angular = 0.5
linear = angular * 0.5

# Reprendre les équations du projet1-2
half_interwheel_distance = interwheel_distance / 2
vel_left  = (linear - (angular * half_interwheel_distance)) / left_wheel_radius
vel_right = (linear + (angular * half_interwheel_distance)) / right_wheel_radius


cmd_drive_msg.drivers[0] = vel_left
cmd_drive_msg.drivers[1] = vel_right

# Envoi des commandes au roues par topic ROS
if SIM:
    cmd_drive_pub_sim.publish(cmd_drive_msg)
else:
    cmd_drive_pub_robot.publish(cmd_drive_msg)

In [13]:
# Enregistrer la trajectoire pendant 20 secondes
start = float(rospy.Time().now().secs)
rate = rospy.Rate(50)
while (float(rospy.Time().now().secs) - start) < 20:
    ros_bag.write('/imu', imu_msg, rospy.Time().now())
    ros_bag.write('/wheel_odom_pose', wheel_pose_msg, rospy.Time().now())
    ros_bag.write('/camera_pose', cam_pose_msg, rospy.Time().now())
    rate.sleep()

In [14]:
# arrêter le robot
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0

# Envoi des commandes au roues par topic ROS
if SIM:
    cmd_drive_pub_sim.publish(cmd_drive_msg)
else:
    cmd_drive_pub_robot.publish(cmd_drive_msg)

In [15]:
# fermer proprement le ROSBAG
ros_bag.close()